# RISS 크롤링
### 파라미터 종류가 많은 경우
- 구글 개발자 도구에서 파라미터를 복사
- 딕셔너리로 만든 후 요청할 때 같이 보냄

In [57]:
import requests
from bs4 import BeautifulSoup

param = {
    'isDetailSearch': 'N',
    'searchGubun': 'true',
    'viewYn': 'OP',
    'strQuery': '패션 인공지능',
    'order': 'DESC',
    'onHanja': 'false',
    'strSort': 'RANK',
    'iStartCount': 0,
    ''
    'fsearchMethod': 'search',
    'sflag': 1,
    'isFDetailSearch': 'N',
    'pageNumber': 1,
    'icate': 're_a_kor',
    'colName': 're_a_kor',
    'pageScale': 10,
    'isTab': 'Y',
    'query': '패션 인공지능'
    }

response = requests.get('https://www.riss.kr/search/Search.do', params=param)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [58]:
contents = soup.select('.srchResultListW > ul >li')

## 헤더가 필요한 경우
서버쪽에서 요청을 거절할 때, 필요한 정보를 같이 날려준다
- 'User-Agent' : 'Mozilla/5.0'
    - 서버에서 브라우저에서 보내는것 처럼 정보를 속이는 방법
- 'Referer' : 'URL 주소값'
    - 어디서부터 발생한 요청인지 보내는것
## SSL 인증 오류
- `requests.get(link, verify=False)`
- 인증 옵션을 꺼버린다.
```python
import urllib3
urllib3.disable_warnings() #waring 메시지 제거

response = requests.get(link, verify=False)
```

In [50]:
import urllib3
urllib3.disable_warnings()
header = {
    'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
for content in contents:
    title =content.select_one('.title > a').text 
    link = 'https://www.riss.kr/' +content.select_one('.title > a').attrs['href']
    
    #상세 페이지 요청
    response = requests.get(link, headers= header , verify=False)
    html =response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 순서를 기준으로 찾는 방법
    press = soup.select_one('.infoDetailL > ul > li:nth-of-type(2) >div').text
    year = soup.select_one('.infoDetailL > ul > li:nth-of-type(5) >div').text
    keywords = soup.select_one('.infoDetailL > ul > li:nth-of-type(7) >div').text.split(';')
    result = []
    # for keyword in keywords:
    #     print(keyword.strip())
    #     result.append(keyword)
    keywords = [keyword.strip() for keyword in keywords]
    print(keywords)
    print(title, link, press, year)


['디지털 트랜스포메이션', '패션산업', '인공지능', 'AI 진화 단계', 'digital transformation', 'fashion business', 'artificial intelligence', 'AI evolutionary stage']
패션 제조 기업의 디지털 트랜스포메이션을 위한 인공지능 솔루션 개발 및 활용 현황 https://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=dcd1a8e873c0ca787ecd42904f0c5d65&keyword=패션 인공지능 한국패션비즈니스학회 2022
['Artificial Intelligence(인공지능)', 'Fashion Industry(패션산업)', 'Fashion Design (패션 디자인)']
패션산업에 반영된 인공지능 사례연구 -패션 소비자 중심으로- https://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9c3313ce986b119147de9c1710b0298d&keyword=패션 인공지능 한국디자인문화학회 2020
['Artificial Intelligence(인공지능)', 'Fashion Styling Service(패션 스타일링 서비스)', 'Stitch Fix(스티치 픽스)', 'Convergence Content(융합 콘텐츠)']
인공지능(AI) 패션 스타일링 서비스 ‘스티치 픽스’의 융합 콘텐츠 연구 https://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=404996b30e5d827bb7998d826d417196&keyword=패션 인공지능 한국전시산업융합연구원 2020
['인공지능', '기술 혁신', '디자인 혁신', '프로세스 혁신', '패션 디자인 프로세스 혁신', 'Artifi

### 텍스트를 기반으로 검색하는경우
`press = soup.find('span', string='발행기관').find_next_sibling().text`
- find() 메서드를 활용해서 특정 단어를 검색해서 결과를 얻는다.
```python
if(soup.find('span', string='주제어')):
        keywords = soup.find('span', string='주제어').find_next_sibling().text.split(';')
```
만일 찾는 키워드가 없는 경우는 if 문으로 None 체크를 실시

In [64]:
import urllib3
urllib3.disable_warnings()
header = {
    'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Referer' : 'https://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%8C%A8%EC%85%98+%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
}
for content in contents:
    title =content.select_one('.title > a').text 
    link = 'https://www.riss.kr/' +content.select_one('.title > a').attrs['href']
    
    #상세 페이지 요청
    response = requests.get(link, headers= header , verify=False)
    html =response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 순서를 기준으로 찾는 방법
    press = soup.find('span', string='발행기관').find_next_sibling().text
    year = soup.find('span', string='발행연도').find_next_sibling().text
    if(soup.find('span', string='주제어')):
        keywords = soup.find('span', string='주제어').find_next_sibling().text.split(';')
        result = []
    # for keyword in keywords:
    #     print(keyword.strip())
    #     result.append(keyword)
        keywords = [keyword.strip() for keyword in keywords]
        print(keywords)
    else:
        keywords = []
    print(title, link, press, year)


['디지털 트랜스포메이션', '패션산업', '인공지능', 'AI 진화 단계', 'digital transformation', 'fashion business', 'artificial intelligence', 'AI evolutionary stage']
패션 제조 기업의 디지털 트랜스포메이션을 위한 인공지능 솔루션 개발 및 활용 현황 https://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=dcd1a8e873c0ca787ecd42904f0c5d65&keyword=패션 인공지능 한국패션비즈니스학회 2022
['Artificial Intelligence(인공지능)', 'Fashion Industry(패션산업)', 'Fashion Design (패션 디자인)']
패션산업에 반영된 인공지능 사례연구 -패션 소비자 중심으로- https://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=9c3313ce986b119147de9c1710b0298d&keyword=패션 인공지능 한국디자인문화학회 2020
['Artificial Intelligence(인공지능)', 'Fashion Styling Service(패션 스타일링 서비스)', 'Stitch Fix(스티치 픽스)', 'Convergence Content(융합 콘텐츠)']
인공지능(AI) 패션 스타일링 서비스 ‘스티치 픽스’의 융합 콘텐츠 연구 https://www.riss.kr//search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=404996b30e5d827bb7998d826d417196&keyword=패션 인공지능 한국전시산업융합연구원 2020
['인공지능', '기술 혁신', '디자인 혁신', '프로세스 혁신', '패션 디자인 프로세스 혁신', 'Artifi